# Fine Tuned DistilBERT model inference

In [1]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
MAX_LEN = 128

# Creating the Neural Network for Fine Tuning

In [4]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        # self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.l1 = DistilBertModel.from_pretrained("distilbert/distilbert-base-multilingual-cased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

# Load model for inference

In [5]:
import os

output_model_filepath = "output/pytorch_distilbert_20240903.bin"
output_vocab_filepath = "output/vocab_distilbert_20240903.bin"

assert os.path.isfile(output_model_filepath)
assert os.path.isfile(output_vocab_filepath)

In [6]:
model = torch.load(output_model_filepath).to(device)
model.eval()
model

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

# Predict

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained(output_vocab_filepath)
tokenizer

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


DistilBertTokenizer(name_or_path='output/vocab_distilbert_20240903.bin', vocab_size=119547, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [13]:
input_text = "그래요, 어? 일단 차를 알겠고요. 저희들이 kb 금융 기회를 kb 저축은행 인데요. 지금 5월 달에 저희들이 대환 대출 플러스 생활의 자금 쪽으로 나온 대출상품 있어요. 어 네, 그리고 이은 2월 이 7.8% 대로 진행하는 부분이고 상환기간이 운행 까지고요. 중도상환 가능하고 수수료 발생이 없이 원리금 균등 분할상환 혹은 만기일시상환 쪽으로 이자만 갚을 해도 되는 부분이에요. 네, 그리고 최근 맥심 대환대출 플러스 생활자금 쪽으로 오 천만원까지 인데 오천만원 검토 다 받으실 거예요? 일단 잘 알고 있구요, 지금? 맞으시죠? 예 고객님, 혹시 지금? 그 사용하고 있는 휴대폰은 본인 이름으로 돼 있는 거 맞으세요? 어? 이 통신사는 skt lg 예요. 예 삼성 스마트폰 맞으세요? 혹시 카톡을 사용하고 계세요? 그러면은요 카톡 추가를 해서 저희들 회사 상하고 제 이름 넣어드릴게요. 왜요? 예, 왜요? 예 고객님, 저한테 전화를 주셔서 통화 중에? 제가 전화를 못 받으면 카톡으로 이 연락을 할 수도 있을 있어서 그런 거 아니에요? 아니에요? 네 전화를 주시면?"
input_text = "플립은 전용보험써야한대 그래서 6300원.. 엥 그런것도 있음? 응 그렇다네..? 개양아치야.. 방금 가입할라고 들어가니까 극게그거지.. 6300원밖에ㅜ없더라 ㅠ"

inputs = tokenizer(input_text, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        token_type_ids=None
    )
    
    

In [14]:
outputs

tensor([[ 4.3840, -4.4510]], device='cuda:0')

In [15]:
result = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
print(f"result:{result}")

result = np.array(result) >= 0.5
print(f"result:{result}")

np.where(result == True)

result:[[0.9876787066459656, 0.011532388627529144]]
result:[[ True False]]


(array([0]), array([0]))

In [16]:
true_indices = np.where(result == True)[1]
true_indices

array([0])

In [17]:
label_index = true_indices.item()
label_index

0

In [34]:
def predict(model, tokenizer, text):
    # device = model.device
    inputs = tokenizer(text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            token_type_ids=None
        )
    
    result = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
    print(f"result:{result}")

    result = np.array(result) >= 0.5
    print(f"result:{result}")
    
    true_indices = np.where(result == True)[1]
    label_index = true_indices.item()
    return label_index

from typing import List
def is_spam(result):
    print(result)
    return "보이스피싱 O" if result == 1 else "보이스피싱 X"
    

In [35]:
result = predict(model, tokenizer, "에 나왔습니다 나오셨어요. 그래? 지금 저희 만나실 직원 이름이 제주 역 대리 입니다. 그래서 뭐 청소년 무역 김영삼 과장님 보내서 왔습니다. 아, 뭐, 김주혁 이렇게 인사할 겁니다. 저 외근 보는 직원들은 다 나이가 좀 어립니다. 그러니까 일부러 우리 고객님 업무 도와드리려고 왔으니까 그래도 안녕하세요 이렇게 인상을 받아 주시구요. 그리고 전화 좀 바꿔주세요. 그리고 수고비 오늘 경비 오 만원 부분도 얘기를 해야 되고 어 그리고 따로 또 지시한 부분도 있고 하니까. 지금 세시 네시 반까지 오늘 일을 끝내야 됩니다. 자 이것저것 또 제가 지시할 부분들이 있습니다. 고객님께서 어 저랑 통화하시면 되고 잠깐만요. 아직 직원 않나요? 뭐 돌려 예 전화 좀 바꿔드릴까요? 네, 네. 여보세요? 예, 김주혁 드리죠. 예 예 금액이 895만 원입니다. 받으면 얼로 이동하면 되구요 어. 다시 고객 좀 바꿔주세요. 네. 네 여보세요. 예. 저희 직원한테 경비 오 만원 뺀 거라도 얘기했으니까 요. 바로 오늘 빠르게 업무 봐서 좀 잘 처리하라고 시켰습니다. 그럼 끊지 마시고 갖고 오 만원 빼서 쓰니깐 천 금액이 895 만원이죠. 이거 지금 전화 끊지 마시고 한번 전달해 주시고요. 확인 한번 해주세요. 다시 뭐라고? 백화점 55 처음 네 그 맞겠죠? 뭐 돌아가는 건데 지금 이제 얼른 보내시면 되구요. 그리고 한 오분만 기다려 주세요. 저희가 이미 차 중동 금액이 얼마인지 또 어느 쪽으로 이동을 해서 일을 봐야 되는 제가 요거 한번 파악을 하고 사장님께 오분내로 연락을 드릴 테니깐요. 고객님 뭐 커피 1잔 더 좀 기다리세요 제가 십 분 연락드리겠습니다. 그리고 지금 하셨고 직원은 얼른 갔다가 대리점 들렸다. 고객님 계신 걸로 다시 올 겁니다. 지금 오늘 시간 없으니까 많이 바쁠 거예요. 거기는 좀 전화 기다려주시면 됩니다. 예.")
is_spam(result)

result:[[0.022625355049967766, 0.9778362512588501]]
result:[[False  True]]
1


'보이스피싱 O'

In [36]:
result = predict(model, tokenizer, "나 어제저녁굶어서 이제다이어트할수있을거같아 ㅎㅎ진짜 진심이야 어제?! 나는 쭈욱.. 당분간은 계속.. 얼마나가나본다 어제도 다행히 저녁 빨리먹어서 본전은했다! 지켜본다 원래운동도같이해야하눈대 할 운동이없내ㅠㅠ")
is_spam(result)


result:[[0.9879758954048157, 0.011418935842812061]]
result:[[ True False]]
0


'보이스피싱 X'

In [37]:
result = predict(model, tokenizer, "문자 보고 연락드리는데 OO저축은행 맞나요? 네 고객님 OO저축은행 상담원 이OO입니다. 무엇을 도와드릴까요? 문자 내용대로 정말로 2.5% 대출이 가능한가요? 네 코로나 19로 어려움에 빠져있는 국민들을 위한 정부특별지원상품이라서 간단한 대출조건만 갖추면 가능합니다. 지금 다른 은행에서 대출 받으신거 있으세요? XX은행에서 5천만원 대출한게 있습니다. 네 그럼 먼저 고객님 신용등급을 조회해야하니 지금보내드리는 문자에 있는 링크를 누르셔서 앱을 설치하세요.")
is_spam(result)

result:[[0.07553404569625854, 0.930599570274353]]
result:[[False  True]]
1


'보이스피싱 O'

In [39]:
result = predict(model, tokenizer, "저한테 돈을 맡기시면 대박치실수 있어요. 이자를 많이 드려요.")
is_spam(result)

result:[[0.4095541536808014, 0.567611813545227]]
result:[[False  True]]
1


'보이스피싱 O'

In [41]:
result = predict(model, tokenizer, "저희쪽에 입금해주시면 대신 납부해 드립니다. 계좌번호 알려드릴게요.")
is_spam(result)

result:[[0.4437057673931122, 0.5266515016555786]]
result:[[False  True]]
1


'보이스피싱 O'